# CM50270 Reinforcement Learning
## Coursework Part 1: Value Iteration

In this exercise, you will implement the value iteration algorithm for three closely related, but different, gridworld environments.

**Total number of marks:** 20 marks.

**What to submit:** Your completed Jupyter notebook (.ipynb file) which should include **all** of your source code. Please **do not change the file name or compress/zip your submission**. Please do not include any identifying information on the files you submit. This coursework will be marked **anonymously**.

**Where to submit:** CM50270 Moodle Page.

You are required to **work individually**. You are welcome to discuss ideas with others but you must design your own implementation and **write your own code**.

**Do not plagiarise**. Plagiarism is a serious academic offence. For details on what plagiarism is and how to avoid it, please visit the following webpage: http://www.bath.ac.uk/library/help/infoguides/plagiarism.html

If you are asked to use specific variable names, data-types, function signatures and notebook cells, **please ensure that you follow these instructions**. Not doing so will cause the automarker to reject your work, and will assign you a score of zero for that question. **If the automarker rejects your work because you have not followed the instructions, you may not get any credit for your work**.

Please **do not use any non-standard, third-party libraries** apart from numpy and matplotlib. **If we are unable to run your code because you have used unsupported external libraries, you may not get any credit for your work.**

Please remember to **save your work regularly**.

Please be sure to **restart the kernel and run your code from start-to-finish** (Kernel → Restart & Run All) before submitting your notebook. Otherwise, you may not be aware that you are using variables in memory that you have deleted.

Your total runtime must be less than **1 minute** on the University's computers. Otherwise, you may not get credit for your work. You can run your code on the university's computers remotely using [UniDesk](https://bath.topdesk.net/tas/public/ssp/content/detail/knowledgeitem?unid=ff3266344c1d4eb2acb227cc9e3e1eee).

## Introduction
In this coursework, you will implement the Value Iteration algorithm to compute an optimal policy for three different (but closely related) Markov Decision Processes. For your reference, the pseudo-code for the Value Iteration algorithm is reproduced below from the textbook (Reinforcement Learning, Sutton & Barto, 2018, pp. 83).

<img src="images/value_iteration.png" style="width: 800px;"/>

Please note the following about the pseudo-code: The set $\mathcal{S}$ contains all non-terminal states, whereas $\mathcal{S}^+$ is the set of all states (terminal and non-terminal). The symbol $r$ represents the immediate reward on transition from state $s$ to the next state $s'$ via action $a$. 

<img src="images/bombs and gold numbers.png" style="width: 300px;" align="left" caption="Figure 1"/>

The three problems you will solve use variants of the gridworld environment shown on the left. You should be familiar with this kind of environment from the lectures. The grid squares in the figure are numbered as shown. In all exercises, the following are true: 

**Actions available:** The agent has four possible actions in each grid square. These are *west*, *north*, *south*, and *east*. If the direction of movement is blocked by a wall (for example, if the agent executes action south at grid square 1), the agent remains in the same grid square. 

**Collecting gold:** On its first arrival at a grid square that contains gold (from a neighbouring grid square), the agent collects the gold. Note that, in order to collect the gold, the agent needs to transition into the grid square (containing the gold) from a different grid square.

**Hitting the bomb:** On arrival at a grid square that contains a bomb (from a neighbouring grid square), the agent activates the bomb. 

**Terminal states:** The game terminates when all gold is collected or when the bomb is activated. In Exercises 1 and 2, you can define terminal states to be grid squares 18 and 23. In Exercise 3, you will need to define terminal state(s) differently.


### Instructions ###
Set parameter $\theta$ to $1 \times 10 ^{-10}$. You can express that as `1e-10` in Python. 

Set all initial state values $V(s)$ to zero.

Do not use discounting (that is, set $\gamma=1$).

Use the following reward function: $-1$ for each navigation action (including when the action results in hitting the wall), an additional $+10$ for collecting each piece of gold, and an additional $-10$ for activating the bomb. For example, the immediate reward for transitioning into a square with gold (from a neighbouring grid square) is $-1 + 10 = +9$. 

## Exercise 1: Deterministic Environment (0 Marks)

In this exercise, the agent is able to move in the intended direction with certainty. For example, if it executes action _north_ in grid square 0, it will transition to grid square 5 with probability 1. In other words, we have a deterministic environment.

Compute the optimal policy using Value Iteration. 

Your need to produce two one-dimensional numpy arrays with names `policy` and `v`. Both arrays should have a length of 25, with the element at index $i$ representing grid cell $i$ (see figure above). Both arrays should be accessible in the "solution cell" below!

The array `policy` should be a numpy array of strings that specifies an optimal action at each grid location. Please use the abbreviations `"n"`, `"e"`, `"s"`, and `"w"` for the four actions. As an example, the value of `policy` at index `0` needs to give `"n"`, if _north_ is an optimal action in cell 0. The policy for a terminal state can be any action. If there are multiple optimal actions from a state, any optimal action will be considered as a correct answer. 

The array `v` should be an array of floats that contains the expected return at each grid square (that is, the state value under the optimal policy).

In [1]:
# Please write your code for Exercise 1 in this cell or in as many cells as you want ABOVE this cell.
# Your code should compute the values of policy and v from scratch when this cell is executed, 
# using the value iteration algorithm. We will mark your coursework by checking the values of 
# the variables policy and v in the hidden test cell further below. Do NOT delete this cell.

# YOUR CODE HERE
import numpy as np

class Gridworld:
    def __init__(self):
        self.num_rows = 5
        self.num_cols = 5
        self.gold_reward = 10
        self.bomb_reward = -10
        self.gold_positions = np.array([23])
        self.bomb_positions = np.array([18])
        self.random_move_probability = 0.2

        self.actions = ["n", "e", "s", "w"]
        self.num_actions = len(self.actions)
        self.num_fields = self.num_cols * self.num_rows

        self.rewards = np.array([-1]*self.num_fields)
        self.rewards[self.bomb_positions] += self.bomb_reward
        self.rewards[self.gold_positions] += self.gold_reward

        self.step = 0
        self.cumulative_reward = 0
        self.agent_position = np.random.randint(0, 5)

    def make_step(self, action_index):
        """
        Given an action, make state transition and observe reward.

        :param action_index: an integer between 0 and the number of actions (4 in Gridworld).
        :return: (reward, new_position)
            WHERE
            reward (float) is the observed reward
            new_position (int) is the new position of the agent
        """
        # Randomly sample action_index if world is stochastic
        action = self.actions[action_index]

        # Determine new position and check whether the agent hits a wall.
        old_position = self.agent_position
        new_position = self.agent_position
        if action == "n":
            candidate_position = old_position + self.num_cols
            if candidate_position < self.num_fields:
                new_position = candidate_position
        elif action == "e":
            candidate_position = old_position + 1
            if candidate_position % self.num_cols > 0:  # The %-operator denotes "modulo"-division.
                new_position = candidate_position
        elif action == "s":
            candidate_position = old_position - self.num_cols
            if candidate_position >= 0:
                new_position = candidate_position
        elif action == "w":  # "LEFT"
            candidate_position = old_position - 1
            if candidate_position % self.num_cols < self.num_cols - 1:
                new_position = candidate_position
        else:
            raise ValueError('Action was mis-specified!')

        # Update the environment state
        self.agent_position = new_position

        # Calculate reward
        reward = self.rewards[self.agent_position]
        return reward, new_position

    def reset(self):
        self.agent_position = np.random.randint(0, 5)

    def is_terminal_state(self):
        # The following statement returns a boolean. It is 'True' when the agent_position
        # coincides with any bomb_positions or gold_positions.
        return self.agent_position in np.append(self.bomb_positions, self.gold_positions)


class TableAgent(object):
    """
    Agent for Value Iteration
    :param: env: environment, here we use Gridworld() as input.
    """
    def __init__(self, env):
        # number of states
        self.s_len = env.num_fields
        # number of actions
        self.a_len = env.num_actions
        # reward for each state
        self.r = list(env.rewards)
        # the policy of each state,default is 0
        self.pi = np.array([0]*self.s_len)
        # actions - states transformation matrix
        self.p = np.zeros([self.a_len, self.s_len, self.s_len], dtype=np.float)


        # calculate the probability of s' after setting s and a.
        for i, action in enumerate(env.actions):
            # As this is not a stochastic environment, the probability of action is 1.
            prob = 1.0
            for src in range(0, self.s_len):
                step = []
                # For the position of gold and boom, there shouldn't be any other movement.
                if src == 23 or src == 18:
                    continue
                # Add probability to actions - states transformation matrix.
                if action == "n":
                    candidate_position = src + env.num_cols
                    if candidate_position >= env.num_fields:
                        candidate_position = src
                    step.append(candidate_position)
                elif action == "e":
                    candidate_position = src + 1
                    if candidate_position % env.num_cols <= 0:  # The %-operator denotes "modulo"-division.
                        candidate_position = src
                    step.append(candidate_position)
                elif action == "s":
                    candidate_position = src - env.num_cols
                    if candidate_position < 0:
                        candidate_position = src
                    step.append(candidate_position)
                elif action == "w":
                    candidate_position = src - 1
                    if candidate_position % env.num_cols >= env.num_cols - 1:
                        candidate_position = src
                    step.append(candidate_position)
                for dst in step:
                    self.p[i, src, dst] += prob
        # value function
        self.value_pi = np.zeros((self.s_len))
        # states - actions function
        self.value_q = np.zeros((self.s_len, self.a_len))
        # discounting param
        self.gamma = 1

def value_iteration(agent, max_iter=-1):
    """
    This function is the main process of value iteration

    :param obj agent: TableAgent
    :param int max_iter: maximum iteration.
    :return (agent.pi,agent.value_pi), agent.pi: policy, agent.value_pi:value
    """
    iteration = 0
    while True:
        iteration += 1
        # save the value function (we need to compare with new one later.)
        new_value_pi = np.zeros_like(agent.value_pi)
        for i in range(0, agent.s_len):
            value_sas = []
            for j in range(0, agent.a_len):
                # for each s and a, calculate value function
                value_sa = np.dot(agent.p[j, i, :], agent.r + agent.gamma * agent.value_pi)
                value_sas.append(value_sa)
            # find the best value funcion from every col.
            new_value_pi[i] = max(value_sas)

        # if the difference between the new value function and old one is less than 1e10, stop iteration.
        diff = np.sqrt(np.sum(np.power(agent.value_pi - new_value_pi, 2)))
        if diff < 1e-10:
            break
        else:
            agent.value_pi = new_value_pi
        if iteration == max_iter:
            break

    #print('Iter {} rounds converge'.format(iteration))
    for i in range(1, agent.s_len):
        for j in range(0, agent.a_len):
            # calculate the states - actions function
            agent.value_q[i, j] = np.dot(agent.p[j, i, :], agent.r + agent.gamma * agent.value_pi)
        # find the maximum action
        max_act = np.argmax(agent.value_q[i, :])
        agent.pi[i] = max_act
    return agent.pi,agent.value_pi

GW = Gridworld()
Agent = TableAgent(GW)
result = value_iteration(Agent)
policy = np.array([Gridworld().actions[x] for x in result[0]])
v = np.array(result[1])


**Example Test Cell**

In the code cell below, we have provided an example of the type of test code that we will use to mark your work. In these tests, we first check that your `policy` and `v` variables are of the correct type, and then check that their values match the solution. In the future, the test code will be hidden from you.

You must not delete or modify test cells in any way - any modifications you do make will be overwritten at run-time.

In [2]:
# DO NOT DELETE OR MODIFY THIS CELL!
# Your code for Exercise 1 is tested here.

import numpy as np

# We're giving you the solution values for Exercise 1, but not telling you how to compute them!
solution_values = [3.0, 4.0, 5.0, 4.0, 5.0,
                   4.0, 5.0, 6.0, 5.0, 6.0,
                   5.0, 6.0, 7.0, 6.0, 7.0,
                   6.0, 7.0, 8.0, 0.0, 8.0,
                   7.0, 8.0, 9.0, 0.0, 9.0]
solution_values = np.array(solution_values)

# We're giving you the solution policy for Exercise 1, but not telling you how to compute it!
solution_policy = [
                   'n', 'n', 'n', 'n', 'n',
                   'n', 'n', 'n', 'n', 'n',
                   'n', 'n', 'n', 'e', 'n',
                   'n', 'n', 'n', 'n', 'n',
                   'e', 'e', 'e', 'n', 'w',]
solution_policy = np.array(solution_policy)

# Check that policy and v are numpy arrays.
assert(isinstance(policy, np.ndarray))
assert(isinstance(v, np.ndarray))

# Check correct shapes of numpy arrays.
assert(policy.shape == (25, ))
assert(v.shape == (25, ))

# Check whether the numpy arrays have the correct data types.
assert(np.issubdtype(policy.dtype, np.unicode_)) # policy.dtype should be '<U1'
assert(np.issubdtype(v.dtype, np.float64))

# Check whether policy contains only "n", "w", "s", or "e" values.
assert(np.all(np.isin(policy, np.array(["n", "w", "s", "e"]))))

# Print student's solution and true solution for easier comparison / spotting of errors.
print("Student's policy:")
print(np.flip(policy.reshape((5, 5)), 0))
print("Solution policy:")
print(np.flip(solution_policy.reshape((5, 5)), 0))

print("Student's v:")
print(np.flip(v.reshape((5, 5)), 0))
print("Solution v:")
print(np.flip(solution_values.reshape((5, 5)), 0))

# Compare policy (only on states that have a single optimal direction).
states_to_check =  np.array([4, 9, 14, 17, 19, 20, 22, 24])
np.testing.assert_array_equal(policy[states_to_check], solution_policy[states_to_check])

# Compare state_values (also for terminal states --- they have to be zero!).
states_to_check = np.delete(np.arange(25), np.array([18, 23]))
np.testing.assert_array_almost_equal(v[states_to_check], solution_values[states_to_check], decimal=3)

Student's policy:
[['e' 'e' 'e' 'n' 'w']
 ['n' 'n' 'n' 'n' 'n']
 ['n' 'n' 'n' 'e' 'n']
 ['n' 'n' 'n' 'n' 'n']
 ['n' 'n' 'n' 'n' 'n']]
Solution policy:
[['e' 'e' 'e' 'n' 'w']
 ['n' 'n' 'n' 'n' 'n']
 ['n' 'n' 'n' 'e' 'n']
 ['n' 'n' 'n' 'n' 'n']
 ['n' 'n' 'n' 'n' 'n']]
Student's v:
[[7. 8. 9. 0. 9.]
 [6. 7. 8. 0. 8.]
 [5. 6. 7. 6. 7.]
 [4. 5. 6. 5. 6.]
 [3. 4. 5. 4. 5.]]
Solution v:
[[7. 8. 9. 0. 9.]
 [6. 7. 8. 0. 8.]
 [5. 6. 7. 6. 7.]
 [4. 5. 6. 5. 6.]
 [3. 4. 5. 4. 5.]]


## Exercise 2: Stochastic Environment (12 Marks)

In this exercise, we introduce stochasticity into the environment. Now, the agent is not always able to execute its actions as intended.

With probability 0.8, the agent moves as intended. However, with probability 0.2, it moves in a random direction.

For example, from grid square 0, if the agent tries to move north, with probability 0.8 the action will work as intended. But with probability 0.2, the agent's motor control system will move it in a random direction (including north). So, it will randomly try to move west, east, north or south with probability 0.05 each. Notice that the total probability of moving to square 5 (as intended) is 0.8 + 0.05 = 0.85.
 
Compute the optimal policy using Value Iteration.

Your value iteration method should output two one-dimensional numpy arrays with names `policy` and `v`, as in Exercise 1.

In [3]:
# Please write your code for Exercise 2 in this cell or in as many cells as you want ABOVE this cell.
# Your code should compute the values of policy and v from scratch when this cell is executed, 
# using the value iteration algorithm. We will mark your coursework by checking the values of 
# the variables policy and v in the hidden test cell further below. Do NOT delete this cell.

# YOUR CODE HERE
import numpy as np


class Gridworld:
    def __init__(self):
        self.num_rows = 5
        self.num_cols = 5
        self.gold_reward = 10
        self.bomb_reward = -10
        self.gold_positions = np.array([23])
        self.bomb_positions = np.array([18])
        self.random_move_probability = 0.2

        self.actions = ["n", "e", "s", "w"]
        self.num_actions = len(self.actions)
        self.num_fields = self.num_cols * self.num_rows

        self.rewards = np.array([-1]*self.num_fields)
        self.rewards[self.bomb_positions] += self.bomb_reward
        self.rewards[self.gold_positions] += self.gold_reward

        self.step = 0
        self.cumulative_reward = 0
        self.agent_position = np.random.randint(0, 5)

    def make_step(self, action_index):
        """
        Given an action, make state transition and observe reward.

        :param action_index: an integer between 0 and the number of actions (4 in Gridworld).
        :return: (reward, new_position)
            WHERE
            reward (float) is the observed reward
            new_position (int) is the new position of the agent
        """
        # Randomly sample action_index if world is stochastic
        action = self.actions[action_index]

        # Determine new position and check whether the agent hits a wall.
        old_position = self.agent_position
        new_position = self.agent_position
        if action == "n":
            candidate_position = old_position + self.num_cols
            if candidate_position < self.num_fields:
                new_position = candidate_position
        elif action == "e":
            candidate_position = old_position + 1
            if candidate_position % self.num_cols > 0:  # The %-operator denotes "modulo"-division.
                new_position = candidate_position
        elif action == "s":
            candidate_position = old_position - self.num_cols
            if candidate_position >= 0:
                new_position = candidate_position
        elif action == "w":  # "LEFT"
            candidate_position = old_position - 1
            if candidate_position % self.num_cols < self.num_cols - 1:
                new_position = candidate_position
        else:
            raise ValueError('Action was mis-specified!')

        # Update the environment state
        self.agent_position = new_position

        # Calculate reward
        reward = self.rewards[self.agent_position]
        return reward, new_position

    def reset(self):
        self.agent_position = np.random.randint(0, 5)

    def is_terminal_state(self):
        # The following statement returns a boolean. It is 'True' when the agent_position
        # coincides with any bomb_positions or gold_positions.
        return self.agent_position in np.append(self.bomb_positions, self.gold_positions)


class TableAgent(object):
    """
    Agent for Value Iteration
    :param: env: environment, here we use Gridworld() as input.
    """
    def __init__(self, env):
        # number of states
        self.s_len = env.num_fields
        # number of actions
        self.a_len = env.num_actions
        # reward for each state, shape=[1,self.s_len]
        self.r = list(env.rewards)
        # the policy of each state,default is 0,shape=[1,self.s_len]
        self.pi = np.array([0]*self.s_len)
        # actions - states transformation matrix ,shape=[self.a_len, self.s_len, self.s_len]
        self.p = np.zeros([self.a_len, self.s_len, self.s_len], dtype=np.float)
        # value function
        self.value_pi = np.zeros((self.s_len))
        # states - actions function
        self.value_q = np.zeros((self.s_len, self.a_len))
        # discounting param
        self.gamma = 1
        # calculate states - actions function for each s and a
        self.fresh_p(env)


    def fresh_p(self,env):
        """
        Calculate states - actions function for each s and a
        :param: env: environment, here we use Gridworld() as input.
        """

        # calculate the probability of s' after setting s and a.
        for i, action in enumerate(env.actions):
            # 2 probabilities of action.
            prob1 = 0.8
            prob2 = 0.05 # (1-0.8)/4 = 0.05
            for src in range(0, self.s_len):
                step = []
                # For the position of gold and boom, there shouldn't be any other movement.
                if src == 23 or src == 18:
                    continue
                # Add probability to actions - states transformation matrix.
                return_v = self.return_step_and_pro(action,src,env,prob1,prob2)
                step = return_v[0]
                pro = return_v[1]
                for k in range(0,len(step)):
                    self.p[i, src, step[k]] += pro[k]

    def return_step_and_pro(self,action,src,env,prob1,prob2):
        """

        :param action: the action intended to do.
        :param src:  current position.
        :param env:  Gridworld.
        :param prob1: the probability of doing the intended action.
        :param prob2: (1-prob1)/4. The probability of randomly doing other actions.
        :return: step: next position after doing action. pro: probability of doing that action.
        """
        step = []
        pro = []
        if action == "n":
            candidate_position = src + env.num_cols
            if candidate_position >= env.num_fields:
                candidate_position = src
        elif action == "e":
            candidate_position = src + 1
            if candidate_position % env.num_cols <= 0:  # The %-operator denotes "modulo"-division.
                candidate_position = src
        elif action == "s":
            candidate_position = src - env.num_cols
            if candidate_position < 0:
                candidate_position = src
        elif action == "w":
            candidate_position = src - 1
            if candidate_position % env.num_cols >= env.num_cols - 1:
                candidate_position = src
        step.append(candidate_position)
        pro.append(prob1)

        candidate_position = src + env.num_cols
        if candidate_position >= env.num_fields:
            candidate_position = src
        step.append(candidate_position)

        candidate_position = src + 1
        if candidate_position % env.num_cols <= 0:  # The %-operator denotes "modulo"-division.
            candidate_position = src
        step.append(candidate_position)

        candidate_position = src - env.num_cols
        if candidate_position < 0:
            candidate_position = src
        step.append(candidate_position)

        candidate_position = src - 1
        if candidate_position % env.num_cols >= env.num_cols - 1:
            candidate_position = src
        step.append(candidate_position)

        pro.extend([prob2]*4)
        return step,pro


def value_iteration(agent, max_iter=-1):
    """
    This function is the main process of value iteration

    :param obj agent: TableAgent
    :param int max_iter: maximum iteration.
    :return (agent.pi,agent.value_pi), agent.pi: policy, agent.value_pi:value
    """
    iteration = 0
    while True:
        iteration += 1
        # save the value function (we need to compare with new one later.)
        new_value_pi = np.zeros_like(agent.value_pi)
        for i in range(0, agent.s_len):
            value_sas = []
            for j in range(0, agent.a_len):
                # for each s and a, calculate value function
                value_sa = np.dot(agent.p[j, i, :], agent.r + agent.gamma * agent.value_pi)
                value_sas.append(value_sa)
            # find the best value funcion from every col.
            new_value_pi[i] = max(value_sas)

        # if the difference between the new value function and old one is less than 1e10, stop iteration.
        diff = np.sqrt(np.sum(np.power(agent.value_pi - new_value_pi, 2)))
        if diff < 1e-10:
            break
        else:
            agent.value_pi = new_value_pi
        if iteration == max_iter:
            break

    for i in range(1, agent.s_len):
        for j in range(0, agent.a_len):
            # calculate the states - actions function
            agent.value_q[i, j] = np.dot(agent.p[j, i, :], agent.r + agent.gamma * agent.value_pi)
        # find the maximum action
        max_act = np.argmax(agent.value_q[i, :])
        agent.pi[i] = max_act
    return agent.pi,agent.value_pi

GW = Gridworld()
Agent = TableAgent(GW)
result = value_iteration(Agent)
policy = np.array([Gridworld().actions[x] for x in result[0]])
v = np.array(result[1])

#print(np.flip(policy.reshape((5, 5)), 0))
#print(np.flip(v.reshape((5, 5)), 0))

[['e' 'e' 'e' 'n' 'w']
 ['n' 'n' 'n' 'n' 'n']
 ['n' 'n' 'n' 'e' 'n']
 ['n' 'n' 'n' 'e' 'n']
 ['n' 'n' 'n' 'n' 'n']]
[[6.04169329 7.28756636 8.61359951 0.         8.69262311]
 [4.86185111 5.99087587 6.37082431 0.         6.46721593]
 [3.67550938 4.69621388 4.99441863 3.2189158  5.10250988]
 [2.48699534 3.40945989 3.66922967 2.64122933 3.78610115]
 [1.35979208 2.19733672 2.42878751 1.57272161 2.55202451]]


In [4]:
# DO NOT DELETE OR MODIFY THIS CELL!
# Your code for Exercise 2 is tested here.


## Exercise 3: Stochastic Environment with Two Pieces of Gold (8 marks)

<img src="images/bomb and two gold.png" style="width: 300px;" align="left" caption="Figure 1"/> In this exercise, we have modified the stochastic environment presented in exercise 2. A second piece of gold has been placed on grid square 12. The terminal state is reached only when **all** pieces of gold are collected or when the bomb is activated.

Compute the optimal policy for this altered environment using Value Iteration.

Hint: You will need to change your state representation in order to account for the additional piece of gold.

Your method should output two one-dimensional numpy arrays with names `policy` and `v`, as in the previous exercises. These arrays should specify the expected return and an optimal policy at the corresponding grid sqaure **before any pieces of gold are collected or a bomb is activated.** 

In [5]:
# Please write your code for Exercise 3 in this cell or in as many cells as you want ABOVE this cell.
# Your code should compute the values of policy and v from scratch when this cell is executed, 
# using the value iteration algorithm. We will mark your coursework by checking the values of 
# the variables policy and v in the hidden test cell further below. Do NOT delete this cell.

# YOUR CODE HERE
import numpy as np


class Gridworld:
    def __init__(self):
        self.num_rows = 5
        self.num_cols = 5
        self.num_dimensions = 3
        self.gold_reward = 10
        self.bomb_reward = -10
        self.gold_positions = np.array([12,23,37,73])
        self.bomb_positions = np.array([18,43,68])
        self.random_move_probability = 0.2
        self.end_positinos = [37,73,18,43,68]

        self.actions = ["n", "e", "s", "w"]
        self.num_actions = len(self.actions)
        self.num_fields = self.num_cols * self.num_rows * self.num_dimensions
        self.num_cells = self.num_cols * self.num_rows

        self.rewards = np.array([-1]*self.num_fields)
        for bomb_position in self.bomb_positions:
            self.rewards[bomb_position] += self.bomb_reward
        for gold_position in self.gold_positions:
            self.rewards[gold_position] += self.gold_reward

        self.step = 0
        self.cumulative_reward = 0
        self.agent_position = np.random.randint(0, 5)

    def make_step(self, action_index):
        """
        Given an action, make state transition and observe reward.

        :param action_index: an integer between 0 and the number of actions (4 in Gridworld).
        :return: (reward, new_position)
            WHERE
            reward (float) is the observed reward
            new_position (int) is the new position of the agent
        """
        # Randomly sample action_index if world is stochastic
        action = self.actions[action_index]

        # Determine new position and check whether the agent hits a wall.
        old_position = self.agent_position
        new_position = self.agent_position
        if action == "n":
            candidate_position = old_position + self.num_cols
            if candidate_position < self.num_fields:
                new_position = candidate_position
        elif action == "e":
            candidate_position = old_position + 1
            if candidate_position % self.num_cols > 0:  # The %-operator denotes "modulo"-division.
                new_position = candidate_position
        elif action == "s":
            candidate_position = old_position - self.num_cols
            if candidate_position >= 0:
                new_position = candidate_position
        elif action == "w":  # "LEFT"
            candidate_position = old_position - 1
            if candidate_position % self.num_cols < self.num_cols - 1:
                new_position = candidate_position
        else:
            raise ValueError('Action was mis-specified!')

        # Update the environment state
        self.agent_position = new_position

        # Calculate reward
        reward = self.rewards[self.agent_position]
        return reward, new_position

    def reset(self):
        self.agent_position = np.random.randint(0, 5)

    def is_terminal_state(self):
        # The following statement returns a boolean. It is 'True' when the agent_position
        # coincides with any bomb_positions or gold_positions.
        return self.agent_position in np.append(self.bomb_positions, self.gold_positions1,self.gold_positions2)


class TableAgent(object):
    """
    Agent for Value Iteration
    :param: env: environment, here we use Gridworld() as input.
    """
    def __init__(self, env):
        # number of states
        self.s_len = env.num_fields
        # number of actions
        self.a_len = env.num_actions
        # reward for each state, shape=[1,self.s_len]
        self.r = list(env.rewards)
        # the policy of each state,default is 0,shape=[1,self.s_len]
        self.pi = np.array([0]*self.s_len)
        # actions - states transformation matrix ,shape=[self.a_len, self.s_len, self.s_len]
        self.p = np.zeros([self.a_len, self.s_len, self.s_len], dtype=np.float)
        # value function
        self.value_pi = np.zeros((self.s_len))
        # states - actions function
        self.value_q = np.zeros((self.s_len, self.a_len))
        # discounting param
        self.gamma = 1
        # calculate states - actions function for each s and a
        self.fresh_p(env)

    def fresh_p(self,env):
        """
        Calculate states - actions function for each s and a
        :param: env: environment, here we use Gridworld() as input.
        """

        # calculate the probability of s' after setting s and a.
        for i, action in enumerate(env.actions):
            # 2 probabilities of action.
            prob1 = 0.8
            prob2 = 0.05 # (1-0.8)/4 = 0.05
            for src in range(0, self.s_len):
                step = []
                # For the position of gold and boom, there shouldn't be any other movement.
                if src in env.end_positinos:
                    continue

                # Add probability to actions - states transformation matrix.
                return_v = self.return_step_and_pro(action,src,env,prob1,prob2)
                step = return_v[0]
                pro = return_v[1]
                for k in range(0,len(step)):
                    self.p[i, src, step[k]] += pro[k]


    def return_step_and_pro(self,action,src,env,prob1,prob2):
        """
        :param action: the action intended to do.
        :param src:  current position.
        :param env:  Gridworld.
        :param prob1: the probability of doing the intended action.
        :param prob2: (1-prob1)/4. The probability of randomly doing other actions.
        :return: step: next position after doing action. pro: probability of doing that action.
        """
        step = []
        pro = []

        if src == 12:
            src += 50
        if src == 23:
            src += 25

        if 0 <= src < 25:
            dimension = 1
        elif 25 <= src < 50:
            dimension = 2
        elif 50 <= src < 75:
            dimension = 3

        if action == "n":
            candidate_position = src + env.num_cols
            if candidate_position >= env.num_cells * dimension:
                candidate_position = src
        elif action == "e":
            candidate_position = src + 1
            if candidate_position % env.num_cols <= 0:  # The %-operator denotes "modulo"-division.
                candidate_position = src
        elif action == "s":
            candidate_position = src - env.num_cols
            if candidate_position < 0+(dimension-1)*env.num_cells:
                candidate_position = src
        elif action == "w":
            candidate_position = src - 1
            if candidate_position % env.num_cols >= env.num_cols - 1:
                candidate_position = src
        step.append(candidate_position)
        pro.append(prob1)

        candidate_position = src + env.num_cols
        if candidate_position >= env.num_cells * dimension:
            candidate_position = src
        step.append(candidate_position)

        candidate_position = src + 1
        if candidate_position % env.num_cols <= 0:  # The %-operator denotes "modulo"-division.
            candidate_position = src
        step.append(candidate_position)

        candidate_position = src - env.num_cols
        if candidate_position < 0+(dimension-1)*env.num_cells:
            candidate_position = src
        step.append(candidate_position)

        candidate_position = src - 1
        if candidate_position % env.num_cols >= env.num_cols - 1:
            candidate_position = src
        step.append(candidate_position)

        pro.extend([prob2]*4)
        return step,pro


def value_iteration(agent, max_iter=-1):
    """
    This function is the main process of value iteration

    :param obj agent: TableAgent
    :param int max_iter: maximum iteration.
    :return (agent.pi,agent.value_pi), agent.pi: policy, agent.value_pi:value
    """
    iteration = 0
    while True:
        iteration += 1
        # save the value function (we need to compare with new one later.)
        new_value_pi = np.zeros_like(agent.value_pi)
        for i in range(0, agent.s_len):
            value_sas = []
            for j in range(0, agent.a_len):
                # for each s and a, calculate value function
                value_sa = np.dot(agent.p[j, i, :], agent.r + agent.gamma * agent.value_pi)
                value_sas.append(value_sa)
            # find the best value funcion from every col.
            new_value_pi[i] = max(value_sas)

        # if the difference between the new value function and old one is less than 1e10, stop iteration.
        diff = np.sqrt(np.sum(np.power(agent.value_pi - new_value_pi, 2)))
        if diff < 1e-10:
            break
        else:
            agent.value_pi = new_value_pi
        if iteration == max_iter:
            break
            
    for i in range(1, agent.s_len):
        for j in range(0, agent.a_len):
            # calculate the states - actions function
            agent.value_q[i, j] = np.dot(agent.p[j, i, :], agent.r + agent.gamma * agent.value_pi)
        # find the maximum action
        max_act = np.argmax(agent.value_q[i, :])
        agent.pi[i] = max_act
    return agent.pi,agent.value_pi

GW = Gridworld()
Agent = TableAgent(GW)
result = value_iteration(Agent)
policy = np.array([Gridworld().actions[x] for x in result[0][:25]])
v = np.array(result[1][:25])

#print(np.flip(policy.reshape((5, 5)), 0))
#print(np.flip(v.reshape((5, 5)), 0))


[['e' 'e' 'e' 'w' 'w']
 ['e' 's' 's' 'n' 'n']
 ['e' 'e' 'n' 'w' 'w']
 ['e' 'n' 'n' 'w' 'w']
 ['n' 'n' 'n' 'n' 'w']]
[[10.65103994 11.79603433 13.00848756  4.28547547 12.97048714]
 [11.1861353  12.32932372 12.5121464   0.         10.61568556]
 [12.28702748 13.59365638  4.99441863 12.41930416 11.19974428]
 [11.17522837 12.35165962 13.59092292 12.28122217 11.051285  ]
 [10.06409806 11.17488271 12.28045984 11.10816476  9.99389366]]


In [6]:
# DO NOT DELETE OR MODIFY THIS CELL!
# Your code for Exercise 3 is tested here.
